# 全世代ポケモンの名前とURLの一覧を取得する

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pickle

In [2]:
url = 'https://www.serebii.net/pokemon/nationalpokedex.shtml'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
dextable = soup.select('table.dextable')[0].select('tr')

In [4]:
# 初期化処理、全世代のポケモンのページURLリスト
url_list = []

In [5]:
for i in range(2, len(dextable), 2):
# for i in range(1620, len(dextable), 2):
# for i in range(1624, 1625, 2):
    pokemon = {}
    pokemon['No'] = dextable[i].select('td')[0].text.strip()
    pokemon['English_name'] = dextable[i].select('td')[3].a.text.strip()
    pokemon['Root_URL'] = dextable[i].select('td')[3].a.attrs['href']

    url = f'https://www.serebii.net{pokemon["Root_URL"]}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    if i < 1620: # 第7世代以前
        tmp_url = soup.select('table.dextab')[1].select('td')[0].a.attrs['href']
        tmp_res = requests.get(f'https://www.serebii.net{tmp_url}')
        tmp_soup = BeautifulSoup(tmp_res.text, 'html.parser')
        gen_elems = tmp_soup.select('table.dextab')[1].select('td')
        pokemon['Gen_URLs'] = [{'Gen': gen_item.text.strip(), 'URL': gen_item.a.attrs['href']} for gen_item in gen_elems]
    else: # 第8世代以降
        gen_elems = soup.select('.hubheader')[0]
        pokemon['Gen_URLs'] = [{'Gen': gen_elems.text.strip(), 'URL': gen_elems.a.attrs['href']}]

    url_list.append(pokemon)

In [ ]:
# ファイルに出力
pickle.dump(url_list, open('./all_gen_pokemon_urls.pkl', 'wb'))